## Import library

In [112]:
import pandas as pd
import httpx
import json
import gzip
import string
import datetime
import random

## Set option pandas

In [113]:
pd.options.mode.use_inf_as_na = True
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)


## Read configuration

In [114]:
with open("./credentials.json") as r:
    config = json.loads(r.read())
minio_config = dict(
    key=config["accessKey"],
    secret=config["secretKey"],
    client_kwargs={
        'endpoint_url': config["url"]
    }
)

with open("./api_config.json") as r:
    config = json.loads(r.read())
api_config = dict(
    url=config["url"],
    token=config["token"],
)

## Function call api

In [115]:
def get_process(httpx, url, headers, topic):
  url = url + f"?topic={topic}"
  with httpx.Client() as client:
    r = client.get(url=url, headers=headers)
    json = r.json()
  if json["ok"] == True and len(json["rows"]) != 0:
    return json["rows"]
  else:
    return {}


def set_process(httpx, url, headers, process_id, status):
  url = url + "/status"
  headers["Content-Type"] = "application/x-www-form-urlencoded"
  data = f"processId={process_id}&status={status}"
  with httpx.Client() as client:
    r = client.put(url=url, headers=headers, data=data)
    json = r.json()
  if (json["ok"] == True):
    return json
  else:
    return {}

## Get process from api

In [116]:
# topic = "service_opd_screening"
topic = "appointment"
url = api_config["url"]
token = api_config["token"]
headers = {"Authorization": "Bearer " + token}
r = get_process(httpx=httpx, url=url, headers=headers, topic=topic)

display(r)

{'process_id': '258d59e1-78c5-4aea-be8c-164c5489fb5b',
 'hospcode': '41124',
 'status': 'PROCESS',
 'topic': 'appointment',
 'path': 'upload/41124/appointment/258d59e1-78c5-4aea-be8c-164c5489fb5b'}

## Set parameter after call api

In [117]:
hospcode = r["hospcode"]
process_id = r["process_id"]
source_path = "s3://mophgw/{}".format(r["path"])
destination_path = "s3://phr/{}".format(r["path"])
local_code_path = "s3://mapcode/{}/localcode".format(hospcode)
std_code_path = "s3://mapcode/{}/stdcode".format(hospcode)

## function to read_csv


In [118]:
def read_csv(path: str = "", filename: str = "", is_prefix: bool = True, prefix_type: str = ""):
  df = pd.read_csv(f"{path}/{filename}", engine="c",
              dtype="str", storage_options=minio_config)
  if is_prefix:
    prefix = filename.removesuffix(".csv.gz")
    if prefix_type == "local":
      df = df.add_prefix(f"_{prefix}_local")
    elif prefix_type == "std":
      df = df.add_prefix(f"_{prefix}_")
  return df

## load transaction file to dataframe
- person

In [119]:
person = read_csv(path=source_path, filename="person.csv.gz",is_prefix=False)

## load localcode files to dataframe
- title, gender, rh_blood_group, blood_group, nationality, marital_status, occupation

In [ ]:
local_config = ["blood_group", "gender", "marital_status",
                "nationality", "rh_blood_group", "title"]
local_df = dict()
for k in local_config:
  local_df[k] = read_csv(path=local_code_path, filename=k +
                         ".csv.gz", is_prefix=True, prefix_type="local")
  display(local_df[k])

## load stdcode file to dataframe
- title, gender, rh_blood_group, blood_group, nationality, marital_status, occupation

In [ ]:
# std_config = ["blood_group", "gender", "marital_status",
#               "nationality", "rh_blood_group", "title"]
std_config = dict(
    blood_group=["blood_group", "blood_groups"],
    gender = ["gender","genders"], 
    marital_status = ["marital_status","marital_statuses"],
    nationality = ["nationality","nationalities"], 
    rh_blood_group = ["rh_blood_group","rh_blood_groups"], 
    title = ["title","titles"]
)
std_df = dict()
for k, v in std_config.items():
  std_df[k] = read_csv(path=std_code_path, filename=v[1] + ".csv.gz", is_prefix=True, prefix_type="std")
  display(std_df[k])

## merge localcode to person

In [ ]:
person_merge_local = dict(
  title = ["title_code","_title_localcode"],
  gender = ["gender","_gender_localcode"],
  rh_blood_group = ["rh_blood_group", "_rh_blood_group_localcode"],
  blood_group = ["blood_group","_blood_group_localcode"],
  nationality = ["nationality_code","_nationality_localcode"],
  marital_status = ["marital_status","_marital_status_localcode"],
)
for k, v in person_merge_local.items():
  person = pd.merge(
      person, local_df[k], how="left", left_on=v[0], right_on=v[1])

display(person)

## merge stdcode to person

In [ ]:
person_merge_std = dict(
    title=["_title_localcode", "_titles_localcode"],
    gender=["_gender_localcode", "_genders_localcode"],
    rh_blood_group=["_rh_blood_group_localcode", "_rh_blood_groups_localcode"],
    blood_group=["_blood_group_localcode", "_blood_groups_localcode"],
    nationality=["_nationality_localcode", "_nationalities_localcode"],
    marital_status=["_marital_status_localcode", "_marital_statuses_localcode"],
)
for k, v in person_merge_std.items():
    person = pd.merge(person, std_df[k], how="left", left_on=f"{v[0]}", right_on=f"{v[1]}")
    # person = person.drop_duplicates(subset=["cid"])
display(person)

In [124]:
# with open("./person_config.json", "w") as fw:
#   fw.write(json.dumps(dict(std=person_merge_std, local=person_merge_local)))